# Library Documentation

See the glossary [here](../models/README.md#Glossary) for documentation references.

# Total Cost Scenario

This notebook provides an overview of estimating total costs of conencting a group of schools.
The overview is split into the following categories:
* **Configuration**: where you can configure a variety of model parameters
* **Running models**: where you can start the model execution
* **Outputs**: where you can examine the results in tables and plots and download them locally

Note that the notebook is interactive.
Namely, you can update the configurations, run the models again, and generate a new set of outputs.
You will need to explicitly re-run the models again after a re-configuration (by clicking the `Run Model` button), and you will need to explicitly re-generate the outputs (by clicking the various `Generate ...` buttons) to see the ouputs updated.

## Configuration

You can find a brief description of the model parameters [below](#Parameters). The interactive dashboard below allows you to configure the scenario of interest and the cost models being run inside that scenario.

In [ ]:
from giga.viz.notebooks.cost_estimation_parameter_input import CostEstimationParameterInput

import warnings
warnings.filterwarnings('ignore')

inputs = CostEstimationParameterInput(local_data_workspace='sample_workspace')
inputs.parameter_input()

## Running the Model

In [ ]:
from IPython.display import display, clear_output
from ipywidgets import Button, Output
from tqdm import tqdm
from IPython.display import clear_output
import sys
import time
import logging

from giga.models.scenarios.scenario_dispatcher import create_scenario
from giga.data.space.model_data_space import ModelDataSpace
from giga.schemas.output import OutputSpace
from giga.viz.notebooks.helpers import run_message
from giga.utils.logging import LOGGER

button = Button(description="Run Model")
output = Output()

output_space = None
data_space = None
config = None

display(button, output)

def on_button_clicked(b):
    with output:
        clear_output(wait=True)
        global config
        global data_space
        global output_space
        verbose = inputs.dashboard_parameters()["verbose"]
        LOGGER.propagate = verbose
        config = inputs.scenario_parameters()
        data_space = ModelDataSpace(inputs.data_parameters())
        output_space = OutputSpace()
        scenario = create_scenario(config, data_space, output_space)
        output_space = scenario.run(progress_bar=verbose)
        run_message()

button.on_click(on_button_clicked)

## Output Tables

The summary table below show aggregated cost statistics across the schools of interest.


In [ ]:
import geopandas as gpd
from ipywidgets import HTML

from giga.viz.notebooks.tables import display_summary_table, output_to_capex_details, plot_cost_breakdown
from giga.viz.notebooks.helpers import output_to_table, button_cb, download_link_frame
from giga.viz.notebooks.maps import show_cost_map, show_electricity_map


def action():
    display_summary_table(output_space)
    
    border_file = 'parameter_workspace/TM_WORLD_BORDERS-0.3/TM_WORLD_BORDERS-0.3.shp'
    borders = gpd.read_file(border_file)
    b = borders[borders["NAME"] == 'Rwanda']
    plot_cost_breakdown(output_space, data_space, b)
    
    display(HTML("<hr><b><font color='#5b8ff0'>Electricity Map</b><br><br>"))
    display(show_electricity_map(data_space))
    
    display(HTML("<hr><b><font color='#5b8ff0'>Cost Map</b><br><br>"))
    display(show_cost_map(data_space, output_space))
    
    table = output_to_table(output_space)
    display(download_link_frame(table))
    
button, click = button_cb("Show Summary Table", action)
button.on_click(click)

The table below shows the complete model outputs by school.

In [ ]:
from IPython.display import display, HTML
from giga.viz.notebooks.helpers import output_to_table, button_cb


def action():
    table = output_to_table(output_space)
    display(HTML('<h2>Cost Table</h2>'), table)
    
button, click = button_cb("Show Full Table", action)
button.on_click(click)

## Parameters

The following parameters can be configured in the model:

* **Scenario**
    * `Cost Scenario` determines which scenario to estimate costs for; either `Minimum Cost` which finds the cheapest technology can be selected, or an individual technology can be selected which will find the costs for just that technology
    * `OpEx Years` determines the number of years that will be considered in the total cost estimates, where total cost is CapEx + OpEx * `OpEx Years`
    * `Bandwidth Demand (Mbps)` determines the expected demand at each school being considered
* **Fiber Model**
    * `Annual cost per Mbps (USD)`: the annual cost of connectivity per Mbps in US Dollars
    * `Cost Per km (USD)` is the average cost of laying fiber lines per km in US Dollars
    * `Maintenance Cost per km (USD)` is the expected annual maintenace cost of new fiber lines in US Dollars
    * `Maximum Connection Length (km)` is the maximum length of an individual fiber connection, if a single fiber connection exceeds this length, it will not be considered feasible
    * `Annual Power Required (kWh)` is the annual power in kWh needed to operate the equipment
    * `Economies of Scale` indicates if an economies of scale approach should be used when estimating the needed length of fiber lines
* **Satellite Model**
    * `Installation Cost (USD)` is the cost of intalling only the technology equipment (no electricity) at the school site
    * `Annual cost per Mbps (USD)`is the annual cost of connectivity per Mbps in US Dollars
    * `Annual Maintenance Cost (USD)` is the annual cost of maintaining the  equipment at the school site
    * `Annual Power Required (kWh)` is the annual power in kWh needed to operate the equipment
* **Cellular Model**
    * `Installation Cost (USD)` is the cost of intalling only the technology equipment (no electricity) at the school site
    * `Annual cost per Mbps (USD)`is the annual cost of connectivity per Mbps in US Dollars
    * `Annual Maintenance Cost (USD)` is the annual cost of maintaining the  equipment at the school site
    * `Annual Power Required (kWh)` is the annual power in kWh needed to operate the equipment
    * `Maximum Cell Tower Range (km)` is the maximum distance from a cell tower that a school can receive internet service
* **Electricity Model**
    * `Cost per kWh (USD)` is the expected average cost of electricity for the schools considered in US Dollars
    * `Solar Panel Install Cost (USD)` is the average cost of installing solar panels for the schools considered in US Dollars
